In [1]:
import subprocess as sb
import sys

In [7]:
result, err = sb.Popen('sudo -S tshark -i ens33 \
                        -T fields -e frame.number -e _ws.col.Info -e tcp.payload -e data \
                        -E header=n -E separator=, -E quote=d -E occurrence=f', 
                        shell=True, stdin=sb.PIPE, stdout=sb.PIPE, stderr=sb.PIPE).communicate(input=b'rocks\n')

In [ ]:
    # cmd = 'tshark -i ens33 -Y "tcp.payload" \
    #         -T fields \
    #         -e frame.time_epoch -e ip.src -e tcp.srcport -e ip.dst -e tcp.dstport -e ip.proto \
    #         -e _ws.col.Info -e tcp.payload \
    #         -E header=n -E separator=, -E quote=d -E occurrence=f \
    #         -a duration:20'
    # process = subprocess.Popen(cmd, stdout=subprocess.PIPE, shell=True)
    # for line in iter(lambda: process.stdout.read(1), b""):
    #     # split_line = line.decode('utf=8').split(',')
    #     print(line.decode('utf-8', 'ignore'), end='')

In [36]:
proc = sb.Popen('cat ./test.log', shell=True, stdout=sb.PIPE, stdin=sb.PIPE)
line = ''
lines = []
for char in iter(lambda: proc.stdout.read(1), b""):
    if char == b'\n':
        lines.append(line)
        line = ''
        continue
    line += char.decode('utf-8', 'ignore')
    # print(line.decode('utf-8'), end='')
    # break on newline
    # if line == b'\n': break

In [37]:
lines

['"1671014261.245161442","10.20.0.249","44336","172.217.25.10","443","6","Application Data","17030300221179dcda22ac4d85222e654fab080508d2e844f8c56384d74a01ac8cf77cd95b4d4f"',
 '"1671014261.245368092","10.20.0.249","44336","172.217.25.10","443","6","Application Data","1703030013914bfaa0bd6a3d68e94acf73c792ff6aaf6f00"',
 '"1671014265.116381156","10.20.0.249","41778","151.101.65.69","443","6","Application Data","170303007f000000000000000f6b404615329ca8e2e2784fc04172d9131fbf3c9e89045479a881cbbc86dea30ce55e43089a9d7b4819c16d943816a4b559da643b8f48fd9bac95586455df54e106688b096ae93284c3c28e458ed98de0c875044f967256e60d0f83f8578fc67f0a85a5161c9a658d6b318f762f8f334ab65195d38bb0be"',
 '"1671014265.366834602","151.101.65.69","443","10.20.0.249","41778","6","Application Data, Application Data","170303059bb52fb7b7d986485b92794b66c16f86b531f9ab6fadacce78faa0bf64522c53cce45ed5e3b9260e31268632939851ff2fbb538915b25636b1cd19988cadf5efe739b5703f396c3baf618526095d06aded39f79bc2bddf5efbc703d1dfc432eaa34a0b9e

In [26]:
line.decode('utf-8')

'\n'

In [22]:
import sqlite3 as sql
import pandas as pd
import os
os.remove('test.db')
con = sql.connect('test.db')
con.execute('''CREATE TABLE capture(
    time_epoch FLOAT,
    ip_src varchar(16),
    port_src VARCHAR(8),
    ip_dst VARCHAR(16),
    port_dst varchar(8),
    data varchar(800)
)''')

df = pd.read_sql_query('SELECT * FROM capture', con)
df

,time_epoch,ip_src,port_src,ip_dst,port_dst,data


In [2]:
import pymongo
# Replace the uri string with your MongoDB deployment's connection string.
conn_str = 'mongodb://10.20.0.118:27017'
# set a 5-second connection timeout
client = pymongo.MongoClient(conn_str, serverSelectionTimeoutMS=5000)
try:
    print(client.server_info())
except Exception:
    print("Unable to connect to the server.")

{'version': '5.0.14', 'gitVersion': '1b3b0073a0b436a8a502b612f24fb2bd572772e5', 'modules': [], 'allocator': 'tcmalloc', 'javascriptEngine': 'mozjs', 'sysInfo': 'deprecated', 'versionArray': [5, 0, 14, 0], 'openssl': {'running': 'OpenSSL 1.1.1  11 Sep 2018', 'compiled': 'OpenSSL 1.1.1  11 Sep 2018'}, 'buildEnvironment': {'distmod': 'ubuntu1804', 'distarch': 'x86_64', 'cc': '/opt/mongodbtoolchain/v3/bin/gcc: gcc (GCC) 8.5.0', 'ccflags': '-Werror -include mongo/platform/basic.h -ffp-contract=off -fasynchronous-unwind-tables -ggdb -Wall -Wsign-compare -Wno-unknown-pragmas -Winvalid-pch -fno-omit-frame-pointer -fno-strict-aliasing -O2 -march=sandybridge -mtune=generic -mprefer-vector-width=128 -Wno-unused-local-typedefs -Wno-unused-function -Wno-deprecated-declarations -Wno-unused-const-variable -Wno-unused-but-set-variable -Wno-missing-braces -fstack-protector-strong -Wa,--nocompress-debug-sections -fno-builtin-memcmp', 'cxx': '/opt/mongodbtoolchain/v3/bin/g++: g++ (GCC) 8.5.0', 'cxxflags'

In [7]:
print(client.list_database_names())
dblist = client.list_database_names()
if "DEV" in dblist:
    print("The database exists.")

['DEV', 'admin', 'config', 'local']
The database exists.


In [35]:
# mylist = [
#   { "_id": 1, "name": "John", "address": "Highway 37"},
#   { "_id": 2, "name": "Peter", "address": "Lowstreet 27"},
# ]

# client['DEV']['flow'].insert_many(mylist)

# client['DEV']['flow'].insert_one({'_id': 0, 'name': 'Test', 'address': 'Highway 37'})
coll = client['DEV']['flow'].find().sort( [("_id", -1)] ).limit(1)
for document in coll:
    document['_id'] = str(int(document['_id']) + 1)
    print(document)
    # collection.insert(document)

{'_id': '1', 'name': 'Test', 'address': 'Highway 37'}


In [23]:
list(client['DEV']['flow'].find({}))

[{'_id': ObjectId('63bd16622b0440639cb11bdb'),
  'name': 'Test',
  'address': 'Highway 37'},
 {'_id': ObjectId('63bd16672b0440639cb11bdc'),
  'name': 'Test',
  'address': 'Highway 37'}]

In [30]:
client['DEV']['flow'].drop()

In [3]:
a ='160301029b0100029703037b12b26242bc8f805c5f0843e877701ad81b6e6cacb158ce4ea024dc5e3947e820d5e799fb3df9880864c3cb0ef66dddf7cfea58c4702179479535593593cc13fb0022130113031302c02bc02fcca9cca8c02cc030c00ac009c013c014009c009d002f00350100022c00000020001e00001b7361666562726f7773696e672e676f6f676c65617069732e636f6d00170000ff01000100000a000e000c001d00170018001901000101000b000201000010000e000c02683208687474702f312e310005000501000000000022000a000804030503060302030033006b0069001d0020f254c29fa7c80ed13b579e227e17d0aa6f84ace29a608d06617771c3f1fd5c5b0017004104428f3604be5cb13b940934c78320b7599033fa92d81d064f781caf303b91ceed28f19df24cc3d9412725a90e4c249ab93c2bf0f941aa508795dfb1aae552f1bf002a0000002b00050403040303000d0018001604030503060308040805080604010501060102030201002d00020101001c000240010029011600f100eb0290999d0855ec03cc44b4d5806076297f0dfdb8733633295b05cdfe2191a95d76cd0e1e1c1bed6b826f1b28dde9e22892db6839cf5abcb0bd477c1de63b3b919627a871e14670639c62b5a433edc38e5f90d72c21ff685f74db5516c8d0846947626546f328bf571ef2a7cf5feb8f563c48baa3216c008dbec443aa541505300392d9f235cf30aed2575fb637743a54b0cf86a0c25d76660fc20c248d352d0b6d1ca5f917a65b915c99f6cd2b018fe00b4321359efd1b9554254d216ea1a6d7af7acdc61d6737d0a1cd540f0d8191d3f79645bc848f8bc3dd609fb313d2bd4d484b476a52a7d9ac4c485bca2c1b3f002120d806e9cc0801fdf168af398f16ad2afe19cfc5cf6de6a5dead4cb0421da9ed7c'
# hex to int
[int(a[i:i+2], 16) for i in range(0, len(a), 2)]

[22,
 3,
 1,
 2,
 155,
 1,
 0,
 2,
 151,
 3,
 3,
 123,
 18,
 178,
 98,
 66,
 188,
 143,
 128,
 92,
 95,
 8,
 67,
 232,
 119,
 112,
 26,
 216,
 27,
 110,
 108,
 172,
 177,
 88,
 206,
 78,
 160,
 36,
 220,
 94,
 57,
 71,
 232,
 32,
 213,
 231,
 153,
 251,
 61,
 249,
 136,
 8,
 100,
 195,
 203,
 14,
 246,
 109,
 221,
 247,
 207,
 234,
 88,
 196,
 112,
 33,
 121,
 71,
 149,
 53,
 89,
 53,
 147,
 204,
 19,
 251,
 0,
 34,
 19,
 1,
 19,
 3,
 19,
 2,
 192,
 43,
 192,
 47,
 204,
 169,
 204,
 168,
 192,
 44,
 192,
 48,
 192,
 10,
 192,
 9,
 192,
 19,
 192,
 20,
 0,
 156,
 0,
 157,
 0,
 47,
 0,
 53,
 1,
 0,
 2,
 44,
 0,
 0,
 0,
 32,
 0,
 30,
 0,
 0,
 27,
 115,
 97,
 102,
 101,
 98,
 114,
 111,
 119,
 115,
 105,
 110,
 103,
 46,
 103,
 111,
 111,
 103,
 108,
 101,
 97,
 112,
 105,
 115,
 46,
 99,
 111,
 109,
 0,
 23,
 0,
 0,
 255,
 1,
 0,
 1,
 0,
 0,
 10,
 0,
 14,
 0,
 12,
 0,
 29,
 0,
 23,
 0,
 24,
 0,
 25,
 1,
 0,
 1,
 1,
 0,
 11,
 0,
 2,
 1,
 0,
 0,
 16,
 0,
 14,
 0,
 12,
 2,
 104,
 50,
 8,
 10

In [4]:
import tomli as toml

toml.load('config.toml')

AttributeError: 'str' object has no attribute 'read'